## Assignment 4
Student: Duncan Ferguson

Student Id: 871641260

Class: Comp 4433

Assignment: Assignment 4

Date 8/9/2021

### Assignment Description
Using free and open source tools, provide a set of choropleth visualizations for each of
the columns containing dates such that the resulting visualizations (48 states only) tell
the story by conveying through color, texture, or both the timelines of achievement of
each milestone/column in the provided dataset.

Missing geoJSON_df are of particular interest in
that when a state has never achieved a given milestone, that should be indicated in a
standout manner such as cross-hatching.

Consider that the publication may be
grayscale. Provide a solution for that as well. Provide the titles, labels, and legends
necessary for clarification. File support is given as follows: SturmCodebook has the
explanation. SturmData is the geoJSON_df CSV.

In [734]:
# Importing the Necessary Files
import json
import pandas as pd
from bokeh.io import output_notebook

file = 'states_geo.json'
file2 = 'StrumData.csv'
file3 = 'states_id.csv'

output_notebook()

Loading BokehJS ...

In [735]:
# Importing the JSON File
with open(file) as geoJSON_df:
    dataset = json.load(geoJSON_df)

In [736]:
# Importing the Strum Dataset
with open(file2) as strum:
    df_2 = pd.read_csv(strum)

df_2.head()

,state,fips,icpsr,debtfree,effectivemwpa,earnings,wills,soletrader
0,AL,1,41,1846.0,NaN,1887.0,NaN,NaN
1,AR,5,42,1835.0,1873.0,1873.0,1868.0,1868.0
2,AZ,4,61,1864.0,1871.0,1973.0,NaN,1871.0
3,CA,6,71,1850.0,1872.0,1872.0,1874.0,1872.0
4,CO,8,62,1861.0,1861.0,1861.0,1874.0,1874.0


In [737]:
# Importing the State IDs
with open(file3) as state_id:
    df_3 = pd.read_csv(file3, usecols=[0, 1], names=['state_id', 'state'], encoding='ascii', sep=',"\t', engine='python')
    df_3['state'] = df_3['state'].str.strip('"')

df_3.head()

,state_id,state
0,0,MN
1,1,MT
2,2,ND
3,3,HI
4,4,ID


In [738]:
import numpy as np

# Merging Data Sets and finding Missing States
strum_w_id = df_2.merge(df_3, left_on='state', right_on='state')  # Merged Data Set
missing_states = np.setdiff1d(df_3['state'].to_list(), strum_w_id['state'].to_list()).tolist()  # Missing States

print(missing_states)

# Display New merged data set
strum_w_id.head()

['AK', 'DC', 'HI']


,state,fips,icpsr,debtfree,effectivemwpa,earnings,wills,soletrader,state_id
0,AL,1,41,1846.0,NaN,1887.0,NaN,NaN,28
1,AR,5,42,1835.0,1873.0,1873.0,1868.0,1868.0,14
2,AZ,4,61,1864.0,1871.0,1973.0,NaN,1871.0,6
3,CA,6,71,1850.0,1872.0,1872.0,1874.0,1872.0,7
4,CO,8,62,1861.0,1861.0,1861.0,1874.0,1874.0,8


In [739]:
# Removing Missing states from the JSON file
datetime_cols = ['debtfree', 'effectivemwpa', 'earnings', 'wills', 'soletrader']  # columns to turn into datetime
for element in reversed(dataset['features']):
    element['state'] = df_3.iloc[element['id'], :][1]  # Add state
    add_df = strum_w_id[strum_w_id['state_id'] == element['id']]
    if element['state'] in missing_states:  # If the state is in that change list, remove it
        dataset['features'].remove(element)  # Removing states that are in the data list
    else:
        for i in add_df:
            element['properties'][i] = add_df[i].values[0]

# Reformatting the difference in the data
for element in dataset['features']:
    for i in element['properties']:
        if i in datetime_cols:
            if np.isnan(element['properties'][i]):
                element['properties'][i] = 'No Data'
            else:
                element['properties'][i] = int(element['properties'][i])

In [740]:
import  geopandas as gpd
from bokeh.models import GeoJSONDataSource

# Creating a GeoDataFrame
j_dataset = gpd.GeoDataFrame.from_features(dataset["features"])
json_data = json.dumps(json.loads(j_dataset.to_json()))
geosource = GeoJSONDataSource(geojson=json_data)

j_dataset.head()

,geometry,scalerank,featurecla,adm1_code,state,fips,icpsr,debtfree,effectivemwpa,earnings,wills,soletrader,state_id
0,"POLYGON ((-89.59941 48.01027, -89.48888 48.013...",2,Admin-1 scale rank,USA-3514,MN,27,33,1866,1869,1869,1869,1874,0
1,"POLYGON ((-111.19419 44.56116, -111.29155 44.7...",2,Admin-1 scale rank,USA-3515,MT,30,64,1864,1887,1887,No Data,1874,1
2,"POLYGON ((-96.60136 46.35136, -96.53891 46.199...",2,Admin-1 scale rank,USA-3516,ND,38,36,1862,1877,No Data,1877,1877,2
3,"POLYGON ((-111.04973 44.48816, -111.05025 42.0...",2,Admin-1 scale rank,USA-3518,ID,16,63,1887,1903,1915,1887,1887,4
4,"POLYGON ((-116.99807 46.33017, -116.90653 46.1...",2,Admin-1 scale rank,USA-3519,WA,53,73,1860,1881,1881,1889,1889,5


Now that we have imported all of the datasets. Scrubbed them. Identified missing states. Removed them from
the Json file and then merged all the relevant information on to the JSON file it is time to start graphing and looking
at the data.

In [741]:
# importing the necessary dependencies
from bokeh.plotting import figure, show
from bokeh.models import ColorBar, LinearColorMapper, HoverTool
from bokeh.palettes import gray, viridis
from ipywidgets import interact, widgets

# Setting Up Color Scale to use
def get_plot(category, value):

    # Changing Color Scale of Map
    if value == 'color':
        palette = viridis(100)
        nan_color = '#d9d9d9'
        line_color = 'red'
    else:
        palette = gray(100)
        nan_color = '#FFFFFF'
        line_color = 'black'
    palette = palette[::-1]  # Making Dark Colors the most recent

    # Adding in added descriptions to the title
    if category == 'debtfree':
        category_description = 'Law protecting married women’s separate property from her husband’s debts'
    elif category == 'effectivemwpa':
        category_description = 'Law granting married women control and management rights over their separate property'
    elif category == 'earnings':
        category_description = 'Law granting married women ownership of their wages or earnings on par with other separate property'
    elif category == 'wills':
        category_description = 'Law granting married women the ability to write wills without their husband\'s consent or other restrictions'
    elif category == 'soletrader':
        category_description = 'Law granting married women as a class the right to sign contracts and engage in business without consent of husband'
    else:
        category_description = 'No Category Selected'

    # Setting Colors for map
    color_mapper = LinearColorMapper(palette=palette,
                                     low=strum_w_id[category].min(),
                                     high=strum_w_id[category].max(),
                                     nan_color=nan_color)

    color_bar = ColorBar(color_mapper=color_mapper,
                         label_standoff=8,
                         title="Year of Adoption",
                         width=20,
                         height=500,
                         border_line_color='black',
                         location=(0, 0),
                         orientation='vertical')

    # Displaying map
    p = figure(title=category + ' by Year: \n' + category_description,
               plot_height=600,
               plot_width=950,
               toolbar_location='right',
               tools='pan,box_select,zoom_in,zoom_out,save,reset',
               y_axis_label='Longitude',
               x_axis_label='Latitude')

    # Blanking out the grid lines
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.patches('xs', 'ys', source=geosource,
              fill_color={'field': category, 'transform': color_mapper},
              line_color=line_color,
              line_width=0.25,
              fill_alpha=1)

    # Adding Colorbar as legend
    p.add_layout(color_bar, 'right')

    # Adding Hover Tool for extra details
    hover = HoverTool()
    hover.tooltips = """
    <div>
        <h4>@state</h3>
        <div><strong>Earnings: </strong>@earnings</div>
        <div><strong>Debtfree: </strong>@debtfree</div>
        <div><strong>Effective_mwpa: </strong>@effectivemwpa</div>
        <div><strong>Earnings: </strong>@earnings</div>
        <div><strong>Wills: </strong>@wills</div>
        <div><strong>Sole_Trader: </strong>@soletrader</div>
    </div>
    """
    p.add_tools(hover)


    return p

In [742]:
# Setting up the options to choose from on the interactive widgets
category_options = ['debtfree', 'effectivemwpa', 'earnings', 'wills', 'soletrader']
value_options = ['color', 'gray']

In [743]:
# Setting up the interaction elements

category_dropdown=widgets.Dropdown(options=category_options,
                                   value='debtfree',
                                   description='Category')

value_radio=widgets.RadioButtons(options=value_options,
                                 description='Color Options')

In [744]:
# creating the interact method
@interact(category=category_dropdown, value=value_radio)
def display_graph(category, value):
    show(get_plot(category, value))

interactive(children=(Dropdown(description='Category', options=('debtfree', 'effectivemwpa', 'earnings', 'will…

## The Data Story

Now that we have created an interactive choropleth we can look at all tof the different time lines of achievement for each
of the columns that were labeled in our data set. The Title changes when the chatergory is changed.
There is an option for a gray scale and all nan_value are displayed as gray. There are also hover values so that
you can get the exact dates and see more details

Debt Free. Analysis. When looking at the debtfree year. A law protecting married women's separate property from
 her husband's debts we can see that Louisiana never adopt the law. Where Arkansas was the earliest to adopt the law and
  Idaho was the last. There is a General trend of the Northwest adopting the law later and the North East Adopting it early one

Effectivmwpa. The Law Granting Married women control and management rights over their separate property. Florida adopted
the law much later than the rest of the US. There is no data for Alabama. Oonce again the North East was earlier to
adopt this law overall.

Earnings. The Law granting married women ownership of their wages or earnings on par with other seperate property.
Maryland Was the Earliest and Arizona was the last With missing data for Oklahoma, New Mexico and North and South Dakota

Wills: Wills have the most missing data. Ohio was the earliest to adopt followed by Connecticut. The rest oif the states
did not adopt the law till much later.

Soletrade: A law granting married women as a class the right to sign contracts and engage in business without
consent of husband. This also has a lot of missing data. Maine being the earliest to adopt, where the rest of the
states in the West adopt much later and the states in the south are null values